# PDF 에서 TEXT 추출 
## - tika 라이브러리 사용
https://pypi.org/project/tika/

설치
pip install tika

In [8]:
import socket
print(socket.gethostname())
hostname = socket.gethostname()

devbox20


In [7]:
!pwd

/home/hyerin/work/repos/10_PDF_Extraction


In [ ]:
from tika import parser
import re
import os

### 파일 이름 설정
    최종적으로 변환되는 파일 -> file_name
   

In [ ]:
file_1 = 'CNN_01.pdf'
file_2 = '08852123.pdf'
file_3 = '2019_uda.pdf'

file_name = file_2

In [9]:
# running: java -jar tika-server1.18.jar before executing code below. 
# parsedPDF = parser.from_file(file_name)
# fulltext = parsedPDF['content']
# metadata_dict = parsedPDF['metadata']
# title = metadata_dict['title']
# author = metadata_dict['Author'] # capturing all the names from lets say 15 pages. Just want it to capture from first page 
# pages = metadata_dict['xmpTPg:NPages']

In [10]:
def extract_pages(filename):
    raw_xml = parser.from_file(filename, xmlContent=True)
    body = raw_xml['content'].split('<body>')[1].split('</body>')[0]
    
    body = body.replace('- <\p>\n <p>', '-\n')
    body = body.replace('-\n', '')#.replace('\n', " ")#.replace("<p>", "\n").replace("</p>", "\n")#.replace("</p> <p>", "\n \n")
    body = body.replace('\n', ' ')
    body = body.replace('</p>', '</p>\n')

    # 문장 사이에 연결되지 않는 공백 제거
    regex_10 = re.compile('\w\s*[\<][\/][p][\>]\s+[\<][p][\>]\s*[0-9(a-z]') 
    what = regex_10.findall(body)
    for index in range(len(what)):
        body = body.replace(what[index], what[index][0]+" "+what[index][len(what[index])-1])

    # 특수문자 제거
    regex_11 = re.compile('[\<][p][\>][(\d+)(...)(...)(. . .)(&lt;)(&gt;)±Π•−µπλ∈Π∂βρ〈〉∑(⊗)⊗={×ηα|+o·θ∗].+[\<][\/][p][\>]')
    what = regex_11.findall(body)
    for index in range(len(what)):
        body = body.replace(what[index], '')
    
    # 문장이 아닌 구문을 제외
    regex_12 = re.compile('[\<][p][\>].*[\/][p][\>]')
    what = regex_12.findall(body)
    regex_12_2 = re.compile('\w\w[\.]')
    regex_12_3 = re.compile('\w\w[\:]')
    for index in range(len(what)):
        # .으로 끝나는 문장 찾기
        find_2 = regex_12_2.findall(what[index])
        if len(find_2) == 0:  # .으로 끝나는 문장이 없는 경우 삭제
            find_3 = regex_12_3.findall(what[index]) 
            if len(find_3) == 0:  # .으로 끝나지는 않지만 : 으로 끝나는 문장 찾기
                body = body.replace(what[index], "")
    
    # 공백라인 제거
    regex_13 = re.compile('[\>]\s+[\<][p]') 
    what = regex_13.findall(body)
    for index in range(len(what)):
        body = body.replace(what[index], '>\n<p' )               

    body = body.replace("</p>", '').replace("<p>", '').replace("<p />", '')
    
    # html 태그 제거 
    regex_14 = re.compile('[\<].*[\>].*[\<][\/].*[\>]')  #re.compile('[\<][d][i][v].*[d][i][v][\>]')
    what = regex_14.findall(body)
    for index in range(len(what)):
        body = body.replace(what[index], '\n' )
    
    # 제거 되지 않은 html 부분 한번 더 제거
    what = regex_14.findall(body)
    for index in range(len(what)):
        print(what[index])
        body = body.replace(what[index], '\n' )
        
    body = body.replace("</div>", '')
        
    text_pages = body.split("""<div class="page">""")[1:]

    return text_pages

In [11]:
separated_pages = extract_pages(file_name)
# print(separated_pages[0])

분리된 페이지 수 확인

In [14]:
print(len(separated_pages))

8


'현재디렉토리/PDF이름/' 디렉토리에 페이지 단위로 txt 파일 생성

In [13]:
new_txt_name = file_name.split(".")[0]
directory = "./"+new_txt_name
try:
    if not os.path.exists(directory):
        os.makedirs(directory)
except OSError:
    print ('Error: Creating directory. ' +  directory)

for page_index in range(len(separated_pages)):
    f = open(directory+"/"+new_txt_name+"_"+str(page_index+1)+".txt", 'w')
    f.write(separated_pages[page_index])
    f.close()
